# Exportatory code to compare CT processing pipelines
    1. Pipelines: ANTS, FreeSurfer, CIVET
    2. Atlas: DKT

In [48]:
# basic imports
import numpy as np
import pandas as pd
import seaborn as sns

# Paths

In [49]:
data_dir = '/home/nikhil/projects/CT_reproduce/code/compare-surf-tools/data/'

ants_file = 'ABIDE_ants_thickness_data.csv'
fs53_file = 'ABIDE_fs5.3_thickness.csv'
fs51_file = 'cortical_fs5.1_measuresenigma_thickavg.csv'  
subject_chk_file = 'subject_check.csv'
pheno_file = 'ABIDE_Phenotype.csv'

# Read data

In [33]:
ants_data = pd.read_csv(data_dir + ants_file, header=2)
fs53_data = pd.read_csv(data_dir + fs53_file)
fs51_data = pd.read_csv(data_dir + fs51_file)
subject_ckh = pd.read_csv(data_dir + subject_chk_file)
pheno = pd.read_csv(data_dir + pheno_file)

In [47]:
subject_ckh.shape, pheno.shape, ants_data.shape, fs51_data.shape, fs53_data.shape

((1112, 7), (1112, 102), (1101, 99), (1112, 74), (976, 74))

# Column names

In [46]:
set(fs51_data.columns)

{'ICV',
 'LSurfArea',
 'LThickness',
 'L_bankssts_thickavg',
 'L_caudalanteriorcingulate_thickavg',
 'L_caudalmiddlefrontal_thickavg',
 'L_cuneus_thickavg',
 'L_entorhinal_thickavg',
 'L_frontalpole_thickavg',
 'L_fusiform_thickavg',
 'L_inferiorparietal_thickavg',
 'L_inferiortemporal_thickavg',
 'L_insula_thickavg',
 'L_isthmuscingulate_thickavg',
 'L_lateraloccipital_thickavg',
 'L_lateralorbitofrontal_thickavg',
 'L_lingual_thickavg',
 'L_medialorbitofrontal_thickavg',
 'L_middletemporal_thickavg',
 'L_paracentral_thickavg',
 'L_parahippocampal_thickavg',
 'L_parsopercularis_thickavg',
 'L_parsorbitalis_thickavg',
 'L_parstriangularis_thickavg',
 'L_pericalcarine_thickavg',
 'L_postcentral_thickavg',
 'L_posteriorcingulate_thickavg',
 'L_precentral_thickavg',
 'L_precuneus_thickavg',
 'L_rostralanteriorcingulate_thickavg',
 'L_rostralmiddlefrontal_thickavg',
 'L_superiorfrontal_thickavg',
 'L_superiorparietal_thickavg',
 'L_superiortemporal_thickavg',
 'L_supramarginal_thickavg',
 

In [43]:
set(ants_data.columns)

{'3rd ventricle',
 '4th ventricle',
 'Brain stem',
 'CSF',
 'Structure Name',
 'Unnamed: 0',
 'cerebellar vermal lobules I-V',
 'cerebellar vermal lobules VI-VII',
 'cerebellar vermal lobules VIII-X',
 'left accumbens area',
 'left amygdala',
 'left basal forebrain',
 'left caudal anterior cingulate',
 'left caudal middle frontal',
 'left caudate',
 'left cerebellum exterior',
 'left cerebellum white matter',
 'left cuneus',
 'left entorhinal',
 'left fusiform',
 'left hippocampus',
 'left inferior lateral ventricle',
 'left inferior parietal',
 'left inferior temporal',
 'left insula',
 'left isthmus cingulate',
 'left lateral occipital',
 'left lateral orbitofrontal',
 'left lateral ventricle',
 'left lingual',
 'left medial orbitofrontal',
 'left middle temporal',
 'left pallidum',
 'left paracentral',
 'left parahippocampal',
 'left pars opercularis',
 'left pars orbitalis',
 'left pars triangularis',
 'left pericalcarine',
 'left postcentral',
 'left posterior cingulate',
 'left p